# Nashville Trash Hauler Data Exploration

This notebook explores service request data from hubNashville related to missed trash pickups. This is the initial data exploration and cleaning portion of the analysis.

## Objectives:
1. Load and explore the dataset
2. Examine data structure and quality
3. Identify missing values
4. Filter for missed pickup records
5. Clean and standardize the data
6. Prepare data for further analysis

## 1. Import Required Libraries

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import contextily as ctx
# / --- IGNORE ---

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load and Explore the Dataset

In [84]:
# Load the dataset
file_path = 'trash_hauler_report.csv'
df = pd.read_csv(file_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Dataset loaded successfully!
Shape: (20226, 11)
Columns: ['Request Number', 'Date Opened', 'Request ', 'Description', 'Incident Address', 'Zip Code', 'Trash Hauler', 'Trash Route', 'Council District', 'State Plan X', 'State Plan Y']


In [85]:
# Display first few rows
print("First 5 rows:")
df.head()

First 5 rows:


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [86]:
# Display last few rows
print("Last 5 rows:")
df.tail()

Last 5 rows:


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
20221,267125,11/01/19,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20222,267126,11/01/19,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1.749711e+06,669201.601569
20223,267130,11/01/19,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1.770293e+06,674936.303809
20224,267134,11/01/19,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1.785225e+06,627146.400187
20225,267137,11/01/19,Trash - Curbside/Alley Missed Pickup,possibly others missed as well,"604 Somerset Ct, Nashville, TN 37217, United S...",37217.0,RED RIVER,2505,29.0,1.781360e+06,637742.006846


## 3. Examine Data Structure and Types

In [87]:
# Basic information about the dataset
print("Dataset Info:")
df.info()
print("\n" + "="*50)
print("\nData Types:")
print(df.dtypes)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 1.7+ MB


Data Types:
Request Number        int64
Date Opened          object
Request              object
Description          object
Incident Address     object
Zip Code            float64
Trash Hauler         o

In [88]:
# Basic statistics for numerical columns
print("Basic Statistics:")
df.describe()

Basic Statistics:


,Request Number,Zip Code,Council District,State Plan X,State Plan Y
count,20226.000000,20151.000000,20177.000000,2.019800e+04,2.019800e+04
mean,149178.986354,37180.621805,18.079100,1.763012e+06,6.595069e+05
std,71837.306889,66.310919,9.917632,7.754951e+05,4.355405e+04
min,25270.000000,37013.000000,1.000000,1.663490e+06,-2.719106e+05
25%,86052.750000,37205.000000,8.000000,1.727831e+06,6.406542e+05
50%,145223.500000,37209.000000,19.000000,1.745845e+06,6.564858e+05
75%,217162.750000,37214.000000,26.000000,1.759491e+06,6.756736e+05
max,267137.000000,37228.000000,35.000000,3.496892e+07,2.204382e+06


In [89]:
# Examine unique values in key categorical columns
categorical_cols = ['Request ', 'Trash Hauler', 'Council District']

for col in categorical_cols:
    if col in df.columns:
        print(f"\nUnique values in '{col}':")
        value_counts = df[col].value_counts(dropna=False)
        print(value_counts)
        print(f"Total unique values: {df[col].nunique()}")
        print("-" * 40)


Unique values in 'Request ':
Request 
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: count, dtype: int64
Total unique values: 4
----------------------------------------

Unique values in 'Trash Hauler':
Trash Hauler
RED RIVER    14395
METRO         3512
WASTE IND     1350
NaN            901
Metro           68
Name: count, dtype: int64
Total unique values: 4
----------------------------------------

Unique values in 'Council District':
Council District
25.0    1236
21.0    1080
23.0    1073
2.0      986
16.0     985
8.0      976
15.0     916
5.0      854
7.0      834
6.0      820
24.0     766
29.0     760
20.0     706
33.0     699
1.0      674
17.0     662
34.0     634
19.0     582
26.0     571
30.0     568
27.0     541
18.0     528
32.0     500
28.0     472
9.0      458
31.0     429
4.0      305
14.0     181
3.0      171
13.0     152
NaN       

## 4. Identify Missing Values

In [90]:
# Check for missing values
print("Missing Values Summary:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})

print(missing_summary[missing_summary['Missing Count'] > 0])

Missing Values Summary:
                  Missing Count  Missing Percentage
Description                  31            0.153268
Incident Address              9            0.044497
Zip Code                     75            0.370810
Trash Hauler                901            4.454662
Trash Route                 947            4.682092
Council District             49            0.242262
State Plan X                 28            0.138436
State Plan Y                 28            0.138436


In [91]:
# Check for empty strings and whitespace-only values
print("Empty Strings and Whitespace Check:")
for col in df.columns:
    if df[col].dtype == 'object':
        empty_strings = (df[col] == '').sum()
        whitespace_only = df[col].str.strip().eq('').sum() if df[col].dtype == 'object' else 0
        if empty_strings > 0 or whitespace_only > 0:
            print(f"{col}: {empty_strings} empty strings, {whitespace_only} whitespace-only")

Empty Strings and Whitespace Check:


## 5. Filter for Missed Pickup Records

In [92]:
# Examine the 'Request ' column more closely to identify missed pickup patterns
print("All unique request types:")
request_types = df['Request '].value_counts()
for request_type, count in request_types.items():
    print(f"{request_type}: {count}")

All unique request types:
Trash - Curbside/Alley Missed Pickup: 15028
Trash - Backdoor: 2629
Trash Collection Complaint: 2312
Damage to Property: 257


## 6. Data Cleaning and Preprocessing

**GOALS:**
1. Standardize column names for easier handling
2. Convert date columns to proper datetime format
3. Handle any data quality issues

In [93]:
# Standardize column names by stripping whitespace and replacing spaces with underscores
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Define the filter for 'Missed Pickup' requests
missed_pickup_mask = df['Request'].str.contains('Missed Pickup', case=False, na=False)

print("Column names cleaned and 'missed_pickup_mask' created.")
print(f"Number of records initially identified as 'Missed Pickup': {missed_pickup_mask.sum()}")

Column names cleaned and 'missed_pickup_mask' created.
Number of records initially identified as 'Missed Pickup': 15028


In [94]:
#Let's also check descriptions for missed pickup keywords
# to ensure we're not missing any records 
description_missed_mask = df['Description'].str.contains(
    'missed|miss', case=False, na=False
)

print(f"Records with 'missed' or 'miss' in description: {description_missed_mask.sum()}")

# Check overlap between request type and description filters
overlap = (missed_pickup_mask & description_missed_mask).sum()
print(f"Overlap between request type and description filters: {overlap}")

# Check records that have 'missed' in description but not in request type
desc_only = description_missed_mask & ~missed_pickup_mask
print(f"Records with 'missed' in description but not 'Missed Pickup' in request type: {desc_only.sum()}")

if desc_only.sum() > 0:
    print("\nSample of description-only missed records:")
    print(df[desc_only][['Request', 'Description']].head())

Records with 'missed' or 'miss' in description: 13030
Overlap between request type and description filters: 10319
Records with 'missed' in description but not 'Missed Pickup' in request type: 2711

Sample of description-only missed records:
                       Request  \
7   Trash Collection Complaint   
11  Trash Collection Complaint   
17  Trash Collection Complaint   
27  Trash Collection Complaint   
35  Trash Collection Complaint   

                                          Description  
7        Trash out on time, miss again Tuesday. ALLEY  
11                         Missed- 4th week in a row.  
17  Curb/Trash miss everyone already gotten pick u...  
27  missed. Neighbor missed as well-say she is at ...  
35  Missed my trash AGAIN.\r\ncheck your records. ...  


In [95]:
# Check actual column names after cleaning
print("Available columns after cleaning:")
print(df.columns.tolist())

# Find the correct date column names
date_opened_col = next((col for col in df.columns if 'opened' in col.lower()), None)
date_closed_col = next((col for col in df.columns if 'closed' in col.lower()), None)

# Convert date columns to datetime objects if they exist
if date_opened_col:
    df[date_opened_col] = pd.to_datetime(df[date_opened_col])
    print(f"Successfully converted column: {date_opened_col}")
else:
    print("Warning: Could not find an 'opened' date column.")

# Apply the filter to create a new DataFrame with only missed pickup records
missed_pickups_df = df[missed_pickup_mask].copy()

print("\n'missed_pickups_df' created.")
print(f"Shape of the new DataFrame: {missed_pickups_df.shape}")
missed_pickups_df.head()

Available columns after cleaning:
['Request_Number', 'Date_Opened', 'Request', 'Description', 'Incident_Address', 'Zip_Code', 'Trash_Hauler', 'Trash_Route', 'Council_District', 'State_Plan_X', 'State_Plan_Y']
Successfully converted column: Date_Opened

'missed_pickups_df' created.
Shape of the new DataFrame: (15028, 11)


,Request_Number,Date_Opened,Request,Description,Incident_Address,Zip_Code,Trash_Hauler,Trash_Route,Council_District,State_Plan_X,State_Plan_Y
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066
8,25330,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1.794534e+06,618749.342732


In [96]:
# Save the cleaned missed pickups data to CSV for further analysis
output_filename = 'missed_pickups_cleaned.csv'
missed_pickups_df.to_csv(output_filename, index=False)

print(f"Cleaned missed pickup data saved to: {output_filename}")
print(f"Records saved: {len(missed_pickups_df):,}")
print(f"Columns saved: {list(missed_pickups_df.columns)}")

Cleaned missed pickup data saved to: missed_pickups_cleaned.csv
Records saved: 15,028
Columns saved: ['Request_Number', 'Date_Opened', 'Request', 'Description', 'Incident_Address', 'Zip_Code', 'Trash_Hauler', 'Trash_Route', 'Council_District', 'State_Plan_X', 'State_Plan_Y']


## Summary

This notebook completed the initial data exploration and cleaning steps:

1. **Data Loading**: Successfully loaded the trash hauler report CSV file
2. **Data Structure**: Examined the dataset structure, data types, and basic statistics
3. **Data Quality**: Identified missing values and data quality issues
4. **Data Filtering**: Filtered for missed pickup records using request type and description
5. **Data Cleaning**: Standardized column names and converted date columns
6. **Data Preparation**: Created a clean dataset ready for further analysis

The cleaned data is now ready for advanced analysis including:
- Chronic problem identification
- Liquidated damages calculation
- Geographic analysis
- Performance metrics by hauler and zip code

In [3]:
# Prepare missed_pickups_df and normalize date columns for geospatial analysis
if 'Request' in df.columns:
    missed_pickup_mask = df['Request'].str.contains('Missed Pickup', case=False, na=False)
    missed_pickups_df = df[missed_pickup_mask].copy()
    # Normalize date columns if present
    for col in missed_pickups_df.columns:
        if 'date' in col.lower() or 'opened' in col.lower() or 'closed' in col.lower():
            try:
                missed_pickups_df[col] = pd.to_datetime(missed_pickups_df[col])
            except Exception:
                pass
    print('missed_pickups_df created and date columns normalized.')
else:
    print('Request column not found in df.')

NameError: name 'df' is not defined

In [2]:
# Basic Geospatial and Heat Map Visualizations
# Make sure latitude and longitude columns exist in missed_pickups_df
if 'latitude' in missed_pickups_df.columns and 'longitude' in missed_pickups_df.columns:
    # Center map on Nashville
    nashville_coords = [36.1627, -86.7816]
    nashville_map = folium.Map(location=nashville_coords, zoom_start=12)
    # Prepare heatmap data
    heat_data = missed_pickups_df[['latitude', 'longitude']].dropna().values.tolist()
    HeatMap(heat_data, radius=8, blur=15).add_to(nashville_map)
    nashville_map.save('missed_pickups_heatmap.html')
    print("Heatmap saved as 'missed_pickups_heatmap.html'.")
else:
    print("Latitude and Longitude columns are missing in missed_pickups_df.")

# Optional: Plot points using geopandas if available
try:
    if 'latitude' in missed_pickups_df.columns and 'longitude' in missed_pickups_df.columns:
        gdf = gpd.GeoDataFrame(
            missed_pickups_df.dropna(subset=['latitude', 'longitude']),
            geometry=gpd.points_from_xy(missed_pickups_df['longitude'], missed_pickups_df['latitude'])
        )
        ax = gdf.plot(figsize=(10, 10), alpha=0.5, color='red')
        ctx.add_basemap(ax, crs=gdf.crs)
        plt.title('Missed Pickups Geospatial Plot')
        plt.show()
except Exception as e:
    print(f"Geopandas plot error: {e}")

NameError: name 'missed_pickups_df' is not defined